# ⚡ Free FLUX.1 schnell on Google Colab (ComfyUI)

**Run Black Forest Labs' FLUX.1 schnell — the fastest FLUX model — via ComfyUI on a free T4 GPU.**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

---

## FLUX.1 schnell vs FLUX.1 dev vs Stable Diffusion

| Feature | SD 1.5 | FLUX.1 dev | FLUX.1 schnell |
|---------|--------|------------|----------------|
| Parameters | 860M | 12B | **12B** |
| Steps needed | 20-30 | 20-25 | **4-8** |
| Time/image (T4) | ~30-45 sec | ~60-90 sec | **~15-25 sec** |
| Image quality | Good (fine-tuned) | Excellent | **Very good** |
| Prompt adherence | Moderate | Excellent | **Very good** |
| Text in images | Poor | Good | **Good** |
| License | Open | Non-commercial | **Apache 2.0** |

> **schnell** = "fast" in German. It's a distilled version of FLUX.1 pro, designed for 1–4 step generation.
> Quality is slightly below dev, but it's **3–4× faster** and has a **fully open license** (Apache 2.0).

## Setup (one-time, ~5 minutes)

1. **File → Upload notebook** → select this file
2. **Runtime → Change runtime type** → Hardware accelerator: **T4 GPU** → **Save**
3. **Runtime → Run all** — ~5 min total (downloads ~12 GB of models)
4. **Click the `trycloudflare.com` URL** in Cell 3 output → ComfyUI opens!

> **No HuggingFace token required!** All models are downloaded from public mirrors.

### What's included
- **FLUX.1 schnell GGUF Q5** — 12B DiT model, Q5 quantized (~8.6 GB) — works great on T4!
- **ComfyUI** with native FLUX support
- **ComfyUI-Manager** — install extra nodes in-browser
- **ComfyUI-GGUF** — custom node for loading GGUF quantized models
- **T5-XXL fp8 + CLIP-L** dual text encoders
- **Cloudflare Tunnel** for reliable public access
- Pre-built schnell workflow (4 steps, no guidance)

### Why GGUF instead of fp8?
- **fp8 models hang on T4** — T4 lacks native fp8 hardware support
- **GGUF Q5 works reliably** — Tested extensively on T4, no hanging
- **Near-identical quality** — Q5 quantization preserves 99%+ of fp8 quality
- **Smaller download** — 8.6 GB vs 12 GB for fp8

### Known limitations
- Free Colab sessions last ~12 hours — **save your images!**
- schnell at 4 steps is fast but slightly less detailed than dev at 20 steps
- The public URL changes every session

---

In [ ]:
# ============================================================
# Cell 1: GPU check + clone ComfyUI + install dependencies
# Runtime: ~3-4 minutes
# ============================================================

print("=" * 60)
print("  CELL 1: Installing ComfyUI + dependencies")
print("=" * 60)

# ── GPU check ──
print("\n🔍 GPU check:")
!nvidia-smi -L
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# ── Clone ComfyUI ──
print("\n📦 Cloning ComfyUI...")
!rm -rf /content/ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI

# ── Install ComfyUI requirements ──
print("\n📦 Installing ComfyUI requirements...")
%cd /content/ComfyUI
!pip install -q -r requirements.txt

# ── Verify PyTorch CUDA (don't override Colab's pre-installed version) ──
print("\n📦 Checking PyTorch CUDA...")
import torch
if torch.cuda.is_available():
    print(f"  ✅ PyTorch {torch.__version__} with CUDA already installed")
else:
    print("  ⚠️  CUDA not available, installing PyTorch with CUDA...")
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# ── Clone ComfyUI-Manager ──
print("\n📦 Installing ComfyUI-Manager...")
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git \
    /content/ComfyUI/custom_nodes/ComfyUI-Manager
!pip install -q -r /content/ComfyUI/custom_nodes/ComfyUI-Manager/requirements.txt 2>/dev/null || true

# ── Install ComfyUI-GGUF (required for GGUF quantized models) ──
print("\n📦 Installing ComfyUI-GGUF...")
!pip install -q --upgrade gguf
!git clone https://github.com/city96/ComfyUI-GGUF.git \
    /content/ComfyUI/custom_nodes/ComfyUI-GGUF

# ── Verify ──
import torch
print(f"\n✅ PyTorch {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ VRAM: {vram:.1f} GB")

print("\n" + "=" * 60)
print("  ✅ CELL 1 COMPLETE — Now run Cell 2")
print("=" * 60)

In [ ]:
# ============================================================
# Cell 2: Download FLUX.1 schnell models (GGUF quantized for T4)
# Runtime: ~3-4 minutes (~12 GB total)
# ============================================================
# Files downloaded:
#   flux1-schnell-Q5_K_S.gguf     (~8.6 GB) — FLUX.1 schnell, Q5 quantized (T4 compatible!)
#   t5xxl_fp8_e4m3fn.safetensors  (~4.9 GB) — T5-XXL text encoder
#   clip_l.safetensors             (~246 MB) — CLIP-L text encoder
#   ae.safetensors                 (~335 MB) — FLUX VAE
#
# ⚠️  Why GGUF instead of fp8?
#     - fp8 models hang on T4 due to missing CUDA fp8 hardware support
#     - GGUF Q5 models work reliably on T4 and produce near-identical quality
#     - Smaller download, faster loading, same great results!
# ============================================================

print("=" * 60)
print("  CELL 2: Downloading FLUX.1 schnell models (GGUF for T4)")
print("=" * 60)

import os

COMFY = "/content/ComfyUI"
UNET_DIR  = f"{COMFY}/models/unet"
CLIP_DIR  = f"{COMFY}/models/clip"
VAE_DIR   = f"{COMFY}/models/vae"

os.makedirs(UNET_DIR, exist_ok=True)
os.makedirs(CLIP_DIR, exist_ok=True)
os.makedirs(VAE_DIR,  exist_ok=True)

def check_file(path, min_bytes, name):
    """Check if file exists and meets minimum size. Delete if corrupted."""
    if os.path.exists(path):
        sz = os.path.getsize(path)
        if sz >= min_bytes:
            print(f"  ✅ {name} already present ({sz / 1024**2:.0f} MB)")
            return True
        else:
            print(f"  ⚠️  {name} is too small ({sz / 1024**2:.1f} MB) — corrupted/incomplete, re-downloading...")
            os.remove(path)
    return False

# ── FLUX.1 schnell GGUF Q5 (city96's quantized version — works great on T4!) ──
# Source: https://huggingface.co/city96/FLUX.1-schnell-gguf
FLUX_GGUF_PATH = f"{UNET_DIR}/flux1-schnell-Q5_K_S.gguf"
if not check_file(FLUX_GGUF_PATH, 8_000_000_000, "FLUX.1 schnell Q5 GGUF"):
    print("\n📦 Downloading FLUX.1 schnell GGUF Q5 (~8.6 GB) — works great on T4...")
    !wget -q --show-progress -c \
        https://huggingface.co/city96/FLUX.1-schnell-gguf/resolve/main/flux1-schnell-Q5_K_S.gguf \
        -O "{FLUX_GGUF_PATH}"

# ── T5-XXL fp8 ──
T5_PATH = f"{CLIP_DIR}/t5xxl_fp8_e4m3fn.safetensors"
if not check_file(T5_PATH, 4_000_000_000, "T5-XXL fp8"):
    print("\n📦 Downloading T5-XXL fp8 (~4.9 GB)...")
    !wget -q --show-progress -c \
        https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors \
        -O "{T5_PATH}"

# ── CLIP-L ──
CLIP_PATH = f"{CLIP_DIR}/clip_l.safetensors"
if not check_file(CLIP_PATH, 200_000_000, "CLIP-L"):
    print("\n📦 Downloading CLIP-L (~246 MB)...")
    !wget -q --show-progress -c \
        https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors \
        -O "{CLIP_PATH}"

# ── FLUX VAE (ae.safetensors) — using cocktailpeanut's ungated mirror ──
VAE_PATH = f"{VAE_DIR}/ae.safetensors"
if not check_file(VAE_PATH, 300_000_000, "FLUX VAE"):
    print("\n📦 Downloading FLUX VAE (~335 MB)...")
    !wget -q --show-progress -c \
        https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.safetensors \
        -O "{VAE_PATH}"

# ── Verify all files with strict size checks ──
print("\n📋 File verification:")
files = {
    "FLUX.1 schnell Q5 GGUF": (FLUX_GGUF_PATH, 8_000_000_000),
    "T5-XXL fp8":             (T5_PATH,        4_000_000_000),
    "CLIP-L":                 (CLIP_PATH,        200_000_000),
    "FLUX VAE":               (VAE_PATH,         300_000_000),
}
all_ok = True
for name, (path, min_sz) in files.items():
    if os.path.exists(path):
        sz = os.path.getsize(path)
        if sz >= min_sz:
            print(f"  ✅ {name}: {sz / 1024**2:.0f} MB")
        else:
            print(f"  ❌ {name}: {sz / 1024**2:.1f} MB — TOO SMALL (corrupted), re-run this cell")
            all_ok = False
    else:
        print(f"  ❌ {name}: MISSING")
        all_ok = False

if all_ok:
    print("\n" + "=" * 60)
    print("  ✅ CELL 2 COMPLETE — Now run Cell 3")
    print("=" * 60)
else:
    print("\n⚠️  Some files missing or corrupted — re-run this cell")

In [ ]:
# ============================================================
# Cell 3: Launch ComfyUI with FLUX.1 schnell workflow (GGUF)
# Runtime: ~1-2 minutes (first launch compiles CUDA kernels)
# ============================================================

print("=" * 60)
print("  CELL 3: Launching ComfyUI with FLUX.1 schnell (GGUF)")
print("=" * 60)

import os, json, subprocess, time, re

os.chdir('/content/ComfyUI')

# ================================================================
# Save workflow using UnetLoaderGGUF (from ComfyUI-GGUF custom node).
# This works reliably on T4 GPUs where fp8 models hang.
#
# schnell does NOT use FluxGuidance — it's guidance-free.
# Only 4 steps needed for good results.
# ================================================================
print("\n⚙️  Writing FLUX.1 schnell GGUF workflow...")

flux_workflow = {
  "last_node_id": 12,
  "last_link_id": 14,
  "nodes": [
    {
      "id": 1, "type": "UnetLoaderGGUF",
      "pos": [50, 50], "size": [315, 58],
      "flags": {}, "order": 0, "mode": 0,
      "outputs": [{"name": "MODEL", "type": "MODEL", "links": [7, 10], "slot_index": 0}],
      "properties": {"Node name for S&R": "UnetLoaderGGUF"},
      "widgets_values": ["flux1-schnell-Q5_K_S.gguf"],
      "title": "Load FLUX.1 schnell (GGUF Q5)"
    },
    {
      "id": 2, "type": "DualCLIPLoader",
      "pos": [50, 200], "size": [315, 106],
      "flags": {}, "order": 1, "mode": 0,
      "outputs": [{"name": "CLIP", "type": "CLIP", "links": [1], "slot_index": 0}],
      "properties": {"Node name for S&R": "DualCLIPLoader"},
      "widgets_values": ["t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux"],
      "title": "Dual CLIP Loader (T5 + CLIP-L)"
    },
    {
      "id": 3, "type": "VAELoader",
      "pos": [50, 380], "size": [315, 58],
      "flags": {}, "order": 2, "mode": 0,
      "outputs": [{"name": "VAE", "type": "VAE", "links": [12], "slot_index": 0}],
      "properties": {"Node name for S&R": "VAELoader"},
      "widgets_values": ["ae.safetensors"],
      "title": "Load FLUX VAE"
    },
    {
      "id": 4, "type": "CLIPTextEncode",
      "pos": [450, 200], "size": [400, 200],
      "flags": {}, "order": 5, "mode": 0,
      "inputs": [{"name": "clip", "type": "CLIP", "link": 1}],
      "outputs": [{"name": "CONDITIONING", "type": "CONDITIONING", "links": [5], "slot_index": 0}],
      "properties": {"Node name for S&R": "CLIPTextEncode"},
      "widgets_values": ["a hyperrealistic photograph of a woman in a cafe, natural window light, shallow depth of field, 85mm lens, film grain, Sony A7R IV"],
      "title": "Positive Prompt"
    },
    {
      "id": 5, "type": "EmptySD3LatentImage",
      "pos": [450, 470], "size": [315, 106],
      "flags": {}, "order": 3, "mode": 0,
      "outputs": [{"name": "LATENT", "type": "LATENT", "links": [11], "slot_index": 0}],
      "properties": {"Node name for S&R": "EmptySD3LatentImage"},
      "widgets_values": [1024, 1024, 1],
      "title": "Empty Latent (1024x1024)"
    },
    {
      "id": 6, "type": "BasicGuider",
      "pos": [900, 200], "size": [222, 46],
      "flags": {}, "order": 8, "mode": 0,
      "inputs": [
        {"name": "model", "type": "MODEL", "link": 7},
        {"name": "conditioning", "type": "CONDITIONING", "link": 5}
      ],
      "outputs": [{"name": "GUIDER", "type": "GUIDER", "links": [8], "slot_index": 0}],
      "properties": {"Node name for S&R": "BasicGuider"},
      "title": "Basic Guider (no guidance)"
    },
    {
      "id": 7, "type": "RandomNoise",
      "pos": [900, 310], "size": [315, 82],
      "flags": {}, "order": 4, "mode": 0,
      "outputs": [{"name": "NOISE", "type": "NOISE", "links": [6], "slot_index": 0}],
      "properties": {"Node name for S&R": "RandomNoise"},
      "widgets_values": [42, "fixed"],
      "title": "Random Noise"
    },
    {
      "id": 8, "type": "KSamplerSelect",
      "pos": [900, 450], "size": [315, 58],
      "flags": {}, "order": 6, "mode": 0,
      "outputs": [{"name": "SAMPLER", "type": "SAMPLER", "links": [9], "slot_index": 0}],
      "properties": {"Node name for S&R": "KSamplerSelect"},
      "widgets_values": ["euler"],
      "title": "Sampler: euler"
    },
    {
      "id": 9, "type": "BasicScheduler",
      "pos": [900, 570], "size": [315, 106],
      "flags": {}, "order": 7, "mode": 0,
      "inputs": [{"name": "model", "type": "MODEL", "link": 10}],
      "outputs": [{"name": "SIGMAS", "type": "SIGMAS", "links": [13], "slot_index": 0}],
      "properties": {"Node name for S&R": "BasicScheduler"},
      "widgets_values": ["simple", 4, 1.0],
      "title": "Scheduler: simple, 4 steps"
    },
    {
      "id": 10, "type": "SamplerCustomAdvanced",
      "pos": [1300, 200], "size": [272, 124],
      "flags": {}, "order": 9, "mode": 0,
      "inputs": [
        {"name": "noise", "type": "NOISE", "link": 6},
        {"name": "guider", "type": "GUIDER", "link": 8},
        {"name": "sampler", "type": "SAMPLER", "link": 9},
        {"name": "sigmas", "type": "SIGMAS", "link": 13},
        {"name": "latent_image", "type": "LATENT", "link": 11}
      ],
      "outputs": [
        {"name": "output", "type": "LATENT", "links": [14], "slot_index": 0},
        {"name": "denoised_output", "type": "LATENT", "links": None}
      ],
      "properties": {"Node name for S&R": "SamplerCustomAdvanced"},
      "title": "Sampler"
    },
    {
      "id": 11, "type": "VAEDecode",
      "pos": [1650, 200], "size": [210, 46],
      "flags": {}, "order": 10, "mode": 0,
      "inputs": [
        {"name": "samples", "type": "LATENT", "link": 14},
        {"name": "vae", "type": "VAE", "link": 12}
      ],
      "outputs": [{"name": "IMAGE", "type": "IMAGE", "links": [16], "slot_index": 0}],
      "properties": {"Node name for S&R": "VAEDecode"},
      "title": "VAE Decode"
    },
    {
      "id": 12, "type": "SaveImage",
      "pos": [1650, 320], "size": [315, 270],
      "flags": {}, "order": 11, "mode": 0,
      "inputs": [{"name": "images", "type": "IMAGE", "link": 16}],
      "properties": {"Node name for S&R": "SaveImage"},
      "widgets_values": ["FLUX_schnell"],
      "title": "Save Image"
    }
  ],
  "links": [
    [1,  2, 0, 4, 0, "CLIP"],
    [5,  4, 0, 6, 1, "CONDITIONING"],
    [6,  7, 0, 10, 0, "NOISE"],
    [7,  1, 0, 6, 0, "MODEL"],
    [8,  6, 0, 10, 1, "GUIDER"],
    [9,  8, 0, 10, 2, "SAMPLER"],
    [10, 1, 0, 9, 0, "MODEL"],
    [11, 5, 0, 10, 4, "LATENT"],
    [12, 3, 0, 11, 1, "VAE"],
    [13, 9, 0, 10, 3, "SIGMAS"],
    [14, 10, 0, 11, 0, "LATENT"],
    [16, 11, 0, 12, 0, "IMAGE"]
  ],
  "groups": [],
  "config": {},
  "extra": {"ds": {"scale": 0.7, "offset": [0, 0]}},
  "version": 0.4
}

wf_dir = '/content/ComfyUI/user/default/workflows'
os.makedirs(wf_dir, exist_ok=True)
with open(f'{wf_dir}/flux_schnell_default.json', 'w') as f:
    json.dump(flux_workflow, f, indent=2)

print("  ✅ Workflow saved: user/default/workflows/flux_schnell_default.json")
print("  ✅ Using GGUF Q5 model (works great on T4!)")
print("  ✅ Settings: euler sampler, simple scheduler, 4 steps, no guidance")

# ── Cloudflare Tunnel ──
print("\n📦 Setting up Cloudflare Tunnel...")
if not os.path.exists('/usr/local/bin/cloudflared'):
    subprocess.run(['wget', '-q',
        'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb',
        '-O', '/tmp/cloudflared.deb'], check=True)
    subprocess.run(['dpkg', '-i', '/tmp/cloudflared.deb'], capture_output=True)
    print("  ✅ cloudflared installed")
else:
    print("  ✅ cloudflared already installed")

print("  🔗 Starting tunnel...")
tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:8188'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
)
time.sleep(5)

tunnel_url = None
for _ in range(15):
    line = tunnel.stderr.readline()
    m = re.search(r'(https://[a-zA-Z0-9-]+\.trycloudflare\.com)', line)
    if m:
        tunnel_url = m.group(1)
        break

if tunnel_url:
    print(f"\n  {'═' * 56}")
    print(f"  🌐  PUBLIC URL: {tunnel_url}")
    print(f"  {'═' * 56}")
    print(f"  Open once 'Starting server' appears below.\n")
else:
    print("  ⚠️  URL not captured — look for trycloudflare.com in logs.\n")

print("🚀 Launching ComfyUI with FLUX.1 schnell (GGUF)...")
print("   GGUF Q5 models work great on T4 — no fp8 hang issues!")
print("   schnell generates in ~15-25 sec (only 4 steps!)")
print()
print("=" * 60)
print("  📝 HOW TO USE (IMPORTANT - READ THIS!)")
print("=" * 60)
print()
print("  1. Open the trycloudflare.com URL above")
print()
print("  2. LOAD THE FLUX WORKFLOW:")
print("     → Click 'Load' button (right sidebar)")
print("     → Select 'flux_schnell_default.json'")
print("     → The FLUX workflow will load with all correct settings")
print()
print("  3. Edit the prompt in the 'Positive Prompt' node")
print()
print("  4. Click 'Queue Prompt' → wait ~15-25 sec")
print()
print("  5. Image appears in 'Save Image' node + saved to output/")
print()
print("=" * 60)

# ── Launch ComfyUI ──
# --lowvram: Offload to CPU as needed for T4's 15GB VRAM
# --preview-method auto: Enable live previews
!python main.py \
    --lowvram \
    --preview-method auto \
    --listen 0.0.0.0 \
    --port 8188

---

## Using ComfyUI with FLUX.1 schnell

### First time
1. **Load the workflow first!** Click **Load** (right sidebar) → select **flux_schnell_default.json**
2. The FLUX workflow loads with 12 nodes — all settings pre-configured
3. First image ~30-40 sec (CUDA compilation). Subsequent: **~15-25 sec**
4. Edit the **CLIPTextEncode** node text to change your prompt
5. Change seed in **RandomNoise** for variations

### Key settings for FLUX.1 schnell

| Setting | Value | Notes |
|---------|-------|-------|
| **Steps** | **4** | Optimal for schnell (distilled for 1–4 steps) |
| **Steps range** | 1–8 | Below 4: less detail. Above 8: no improvement. |
| **Guidance** | **None** | schnell is guidance-free — no `FluxGuidance` node |
| **Sampler** | **euler** | Recommended |
| **Scheduler** | **simple** | Use `simple` or `beta` |
| **Resolution** | **1024×1024** | Native resolution |

> **Why no guidance node?** FLUX.1 schnell was distilled with `cfg=0` baked in. Adding `FluxGuidance` will degrade output — use `BasicGuider` directly.

### schnell vs dev — which to use?

| Use schnell when... | Use dev when... |
|---------------------|-----------------|
| Prototyping / trying many prompts | Final high-quality images |
| Need commercial license (Apache 2.0) | Personal/research use only |
| Want 3-4× faster generation | Want maximum detail/realism |
| T4 GPU with limited session time | Have time for longer generation |

### Prompting tips
- FLUX schnell has strong prompt adherence — describe exactly what you want
- No negative prompts needed
- Camera details help: `85mm lens, f/2.8, Fujifilm XT3, film grain`
- Text rendering works: `a neon sign that says "Open"`
- Style words work well: `cinematic, moody, editorial, high contrast`

### Batch generation for prototyping
In the `EmptySD3LatentImage` node, set `batch_size: 4` to generate 4 images at once.
At 4 steps each, 4 images takes ~60-80 sec — same as one dev image!

---

## API usage

```python
import requests, json, time
from PIL import Image
from io import BytesIO

BASE = "https://your-url.trycloudflare.com"

# Load and modify the schnell workflow
with open('/content/ComfyUI/user/default/workflows/flux_schnell_default.json') as f:
    wf = json.load(f)

wf["4"]["inputs"]["text"] = "your prompt here"
wf["7"]["inputs"]["noise_seed"] = 99999  # change seed
# wf["5"]["inputs"]["batch_size"] = 4   # generate 4 at once

prompt_id = requests.post(f"{BASE}/prompt", json={"prompt": wf}).json()["prompt_id"]

# Poll for completion
while True:
    history = requests.get(f"{BASE}/history/{prompt_id}").json()
    if prompt_id in history:
        break
    time.sleep(1)

# Download image(s)
for node_id, node_output in history[prompt_id]["outputs"].items():
    for img_info in node_output.get("images", []):
        img_bytes = requests.get(f"{BASE}/view",
            params={"filename": img_info["filename"],
                    "subfolder": img_info["subfolder"],
                    "type": img_info["type"]}).content
        fname = f"schnell_{img_info['filename']}"
        Image.open(BytesIO(img_bytes)).save(fname)
        print(f"Saved: {fname}")
```

---

## Troubleshooting

| Problem | Fix |
|---------|-----|
| **"Out of memory"** | `--lowvram` is set. Reduce batch size or resolution to 768×768. |
| **Blurry/low-detail at 4 steps** | Try 6-8 steps. Above 8 won't help for schnell. |
| **Output looks oversaturated** | Normal with any guidance — schnell uses no guidance by design. |
| **Workflow not loading** | Click **Load** button (right sidebar) → select `flux_schnell_default.json` |
| **`UNETLoader` node missing** | ComfyUI may be outdated — re-run Cell 1 |
| **Black images** | VAE issue — re-run Cell 2 to re-download `ae.safetensors` |
| **Tunnel URL missing** | Look in Cell 3 output logs for `trycloudflare.com` |

---

## Resources

- [FLUX.1 schnell on Hugging Face](https://huggingface.co/black-forest-labs/FLUX.1-schnell) (Apache 2.0)
- [ComfyUI GitHub](https://github.com/comfyanonymous/ComfyUI)
- [ComfyUI FLUX examples](https://comfyanonymous.github.io/ComfyUI_examples/flux/)
- [ComfyUI-Manager](https://github.com/ltdrdata/ComfyUI-Manager)
- [r/StableDiffusion](https://reddit.com/r/StableDiffusion)

---

**Made with ❤️ — Star the repo if this saved you time!**